In [26]:
import re
import string
import pandas as pd
from functools import reduce
from math import log

In [27]:
data = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/raw_data/train.csv', header = None)
# df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)
# df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)
data = data.rename({0: 'label', 1: 'text'}, axis = 1)

In [28]:
corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
""".split("\n")[1:-1]

In [29]:
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)


['simple', 'example', 'with', 'cats', 'and', 'mouse']
['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats']
['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


In [30]:
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'cheese', 'mouse', 'with', 'cats', 'example', 'and', 'dogs', 'another', 'simple'}


In [31]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,cheese,mouse,with,cats,example,and,dogs,another,simple
0,0,1,1,1,1,1,0,0,1
1,0,0,1,1,1,1,1,1,1
2,1,1,1,0,1,1,0,1,1


In [32]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf


In [33]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

In [34]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [35]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])


In [36]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [37]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [38]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])


,cheese,mouse,with,cats,example,and,dogs,another,simple
0,0.000000,0.067578,0.0,0.067578,0.0,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.0,0.057924,0.0,0.0,0.156945,0.057924,0.0
2,0.156945,0.057924,0.0,0.000000,0.0,0.0,0.000000,0.057924,0.0


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [40]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
""".split("\n")[1:-1]

In [41]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

TfidfVectorizer(preprocessor=<function preprocessing at 0x392b53d90>)

In [42]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)


In [50]:
tfidf

<13x93 sparse matrix of type '<class 'numpy.float64'>'
	with 140 stored elements in Compressed Sparse Row format>

In [43]:
kmeans = KMeans(n_clusters=2).fit(tfidf)


KMeans(n_clusters=2)

In [44]:
lines_for_predicting = ["tf and idf is awesome!", "some androids is there"]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([1, 0], dtype=int32)